In [1]:
import requests

In [2]:
prem_data2425 = requests.get("https://fbref.com/en/comps/9/Premier-League-Stats")

In [3]:
from bs4 import BeautifulSoup

In [4]:
soup2425 = BeautifulSoup(prem_data2425.text)
standings_table2425 = soup2425.select('table.stats_table')[0]
links2425 = standings_table2425.find_all('a')
links2425 = [l.get("href") for l in links2425]
links2425 = [l for l in links2425 if '/squads/' in l]

IndexError: list index out of range

In [ ]:
team_urls2425 = [f"https://fbref.com{l}" for l in links2425]

In [ ]:
prem_data2425 = requests.get(team_urls2425[0])

In [ ]:
import pandas as pd
import io

html_content2425 = io.StringIO(prem_data2425.text)
matches2425 = pd.read_html(html_content2425, match="Scores & Fixtures")[0]

matches2425.head()

In [ ]:
soup2425 = BeautifulSoup(prem_data2425.text)
links2425 = soup2425.find_all('a')
links2425 = [l.get("href") for l in links2425]
links2425 = [l for l in links2425 if l and 'all_comps/shooting/' in l]

In [ ]:
prem_data2425 = requests.get(f"https://fbref.com{links2425[0]}")

In [ ]:
html_content2425 = io.StringIO(prem_data2425.text)
shooting2425 = pd.read_html(html_content2425, match="Shooting")[0]

shooting2425.head()

In [ ]:
shooting2425.columns = shooting2425.columns.droplevel()

In [ ]:
team_data2425 = matches2425.merge(shooting2425[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [ ]:
years = list(range(2025, 2020, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(10)